In [3]:
from pydub import AudioSegment
import numpy as np

audio = AudioSegment.from_mp3("sample.mp3")
samples = np.array(audio.get_array_of_samples())
samples,len(samples)

FileNotFoundError: [Errno 2] No such file or directory: 'sample.mp3'